In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch

from libs.TrafficGenerator import DatasetConvertor 
from libs.TrafficPredictor.ContextAssisted import PreparingDataset as PreparingDatasetCA
from libs.TrafficPredictor.ContextAssisted import trainModelByDefaultSetting as trainModelCA
from libs.TrafficPredictor.ContextAssisted import evaluateModel as evaluateModelCA
from libs.TrafficPredictor.ContextAssisted import createModel as createModelCA
#from libs.TrafficPredictor.ContextFree import PreparingDataset as PreparingDatasetCF
#from libs.TrafficPredictor.ContextFree import trainModelByDefaultSetting as trainModelCF
#from libs.TrafficPredictor.ContextFree import evaluateModel as evaluateModelCF
#from libs.TrafficPredictor.ContextFree import createModel as createModelCF
from libs import encode_float_filename, decode_float_filename

In [2]:
stringFile = encode_float_filename(0.5)
floatFile = decode_float_filename(stringFile)
print(stringFile)
print(floatFile)

5_00e-01.txt
0.5


In [3]:
dbParams = 0.01
direction = "forward"
#dbParams = 0.15
#direction = "backward"
mode = "fixed"
#mode = "adaptive"
alpha = 0.5
lenWindow_list = [10]
train_ratio = 0.8

rawDatasetFolder = "Dataset"
datasetConverter = DatasetConvertor(rawDatasetFolder)
datasetConverter.processDataset(dbParameter=dbParams, alpha=alpha, mode=mode, direction=direction)
thumbUnit = datasetConverter.getDataUnit('thumb_fr')
#thumbUnit = datasetConverter.getDataUnit('thumb_bk')

========== thumb ============
Forward: Compression rate:0.22314470485360394
========== index ============
Forward: Compression rate:0.24555334544091786
========== middle ============
Forward: Compression rate:0.24907031047306064


In [4]:
for lenWindow in lenWindow_list:
    print(f"================== lenWindow = {lenWindow} =================")
    trainDataAugment = True
    verbose = True

    paramsCF = {}
    paramsCF['lenWindow'] = lenWindow
    paramsCF['lenSource'] = 10
    paramsCF['lenTarget'] = 1
    paramsCF['trainRatio'] = train_ratio
    paramsCF['dataAugment'] = trainDataAugment

    trainData, testData = PreparingDatasetCF(thumbUnit, paramsCF, verbose=verbose)

    bestWeightsCF, _, _, modleParametersCF = trainModelCF(
    paramsCF['lenSource'], paramsCF['lenTarget'], trainData, testData, verbose=verbose)

    trafficModelCF, _ = createModelCF(modleParametersCF)
    trafficModelCF.load_state_dict(bestWeightsCF)

    paramsCF['dataAugment'] = False
    trainDataValid, testDataValid = PreparingDatasetCF(thumbUnit, paramsCF, verbose=verbose)
    actual_trafficCF_train, predicted_trafficCF_train = evaluateModelCF(trafficModelCF, trainDataValid)
    resultsTrain = {}
    resultsTrain['trafficTarget_actual'] = actual_trafficCF_train
    resultsTrain['trafficTarget_predicted'] = predicted_trafficCF_train
    actual_trafficCF_test, predicted_trafficCF_test = evaluateModelCF(trafficModelCF, testDataValid)
    resultsTest = {}
    resultsTest['trafficTarget_actual'] = actual_trafficCF_test
    resultsTest['trafficTarget_predicted'] = predicted_trafficCF_test

    plt.plot(resultsTest['trafficTarget_actual'][0:200])
    plt.plot(resultsTest['trafficTarget_predicted'][0:200])
    plt.show()

    #============= Save Results =============
    dbpString = encode_float_filename(dbParams)
    with open(f"Results/TrafficPredictor/evaluate/CF/{direction}_{lenWindow}_{mode}_{dbpString}_train.pkl", "wb") as file:
        pickle.dump(resultsTrain, file)
    with open(f"Results/TrafficPredictor/evaluate/CF/{direction}_{lenWindow}_{mode}_{dbpString}_test.pkl", "wb") as file:
        pickle.dump(resultsTest, file)

================== lenWindow = 10 =================
Train size: 83253, Test size: 20814
Size of train loader: 21, Size of test loader: 6
Used device: cuda
Size of model: 46969
Seq2Seq(
  (encoder): Encoder(
    (lstm_layers): ModuleList(
      (0): LSTM(1, 12, bidirectional=True)
      (1-5): 5 x LSTM(24, 12, bidirectional=True)
    )
    (ln_layers): ModuleList(
      (0-5): 6 x LayerNorm((24,), eps=1e-05, elementwise_affine=True)
    )
    (residual_projections): ModuleList(
      (0): Linear(in_features=1, out_features=24, bias=True)
      (1-5): 5 x None
    )
  )
  (decoder): Decoder(
    (lstm_layers): ModuleList(
      (0): LSTM(1, 24)
      (1-5): 5 x LSTM(24, 24)
    )
    (ln_layers): ModuleList(
      (0-5): 6 x LayerNorm((24,), eps=1e-05, elementwise_affine=True)
    )
    (residual_projections): ModuleList(
      (0): Linear(in_features=1, out_features=24, bias=True)
      (1-5): 5 x None
    )
    (fc_out): Linear(in_features=24, out_features=1, bias=True)
  )
)
Epoch [1/

KeyboardInterrupt: 

In [5]:
for lenWindow in lenWindow_list:
    print(f"================== lenWindow = {lenWindow} =================")
    trainDataAugment = True
    verbose = True

    paramsCA = {}
    paramsCA['lenSource'] = lenWindow
    paramsCA['lenTarget'] = lenWindow
    paramsCA['trainRatio'] = train_ratio
    paramsCA['dataAugment'] = trainDataAugment
    paramsCA['smoothFc'] = 1.5
    paramsCA['smoothOrder'] = 3

    #============= Training =============
    trainData, testData = PreparingDatasetCA(thumbUnit, paramsCA, verbose=verbose)
    (sources, _, lastTranmittedContext, _, _, _, _, _) = trainData
    print(sources.shape)
    print(lastTranmittedContext.shape)
    bestWeightsCA, _, _, modleParametersCA = trainModelCA(
    paramsCA['lenSource'], paramsCA['lenTarget'], trainData, testData, verbose=verbose)

    #============= Evaluatation =============
    trafficModelCA, _ = createModelCA(modleParametersCA)
    trafficModelCA.load_state_dict(bestWeightsCA)
    paramsValid = paramsCA.copy()
    paramsValid['dataAugment'] = False
    trainData, testData = PreparingDatasetCA(thumbUnit, paramsValid, verbose=verbose)
    resultsTrain = evaluateModelCA(trafficModelCA, trainData)
    resultsTest = evaluateModelCA(trafficModelCA, testData)

    #print(f"actual_class_shape: {resultsTrain['classDistribu_actual'].shape}, predicted_class_shape: {resultsTrain['classDistribu_predicted'].shape}")

    plt.plot(resultsTest['trafficTarget_actual'][0:200])
    plt.plot(resultsTest['trafficTarget_predicted'][0:200])
    plt.show()

    #============= Save Results =============
    dbpString = encode_float_filename(dbParams)
    with open(f"Results/TrafficPredictor/evaluate/CA/{direction}_{lenWindow}_{mode}_{dbpString}_train.pkl", "wb") as file:
        pickle.dump(resultsTrain, file)
    with open(f"Results/TrafficPredictor/evaluate/CA/{direction}_{lenWindow}_{mode}_{dbpString}_test.pkl", "wb") as file:
        pickle.dump(resultsTest, file)
    with open(f"Results/TrafficPredictor/modelParams/{direction}_{lenWindow}_{mode}_{dbpString}.pkl", "wb") as file:
        pickle.dump(modleParametersCA, file)

================== lenWindow = 10 =================
Train size: 83253, Test size: 20814
(83233, 10, 3)
(83233, 1, 3)
Size of train loader: 11, Size of test loader: 3
Used device: cuda
Size of model: 19661
TrafficPredictorContextAssisted(
  (dbf2traffic): DeadFeaturesToTrafficLayer(
    (input_layer): Sequential(
      (0): Linear(in_features=66, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.8, inplace=False)
    )
    (hidden_layers): ModuleList(
      (0-2): 3 x Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.8, inplace=False)
      )
    )
    (trans2transmission_layer): Linear(in_features=64, out_features=10, bias=True)
    (sigmoid): Sigmoid()
    (trans2traffic_layer): Linear(in_features=74, out_features=1

KeyboardInterrupt: 